# Trading Model Development

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set display options
pd.set_option('display.max_columns', None)
sns.set_theme(style='darkgrid')
%matplotlib inline

# Load data
orders_df = pd.read_csv('../data/Orders.csv')

## 2. Data Exploration and Analysis

In [ ]:
# Basic information about the dataset
print("Dataset Info:")
orders_df.info()

print("\nSample Data:")
orders_df.head()

print("\nBasic Statistics:")
orders_df.describe()

# Check for missing values
print("\nMissing Values:")
orders_df.isnull().sum()

## 3. Feature Engineering

In [ ]:
def create_features(df):
    features = pd.DataFrame()
    
    # Time-based features
    fill_times = pd.to_datetime(df['Fill Time'])
    features['hour'] = fill_times.dt.hour
    features['weekday'] = fill_times.dt.dayofweek
    features['is_morning'] = fill_times.dt.hour < 12
    
    # Price features
    features['price_diff'] = df['avgPrice'] - df['Limit Price']
    features['price_ratio'] = df['avgPrice'] / df['Limit Price'].where(df['Limit Price'] != 0, df['avgPrice'])
    
    # Volume features
    features['fill_ratio'] = df['filledQty'] / df['Quantity']
    
    return features

features_df = create_features(orders_df)

## 4. Model Development

In [ ]:
class TradingModel(nn.Module):
    def __init__(self, input_size):
        super(TradingModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.layers(x)

## 5. Training Setup

In [ ]:
# Prepare data
X_train, X_test, y_train, y_test = train_test_split(features_df, targets, test_size=0.2)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create dataloaders
train_loader = DataLoader(
    TradingDataset(X_train_scaled, y_train),
    batch_size=32,
    shuffle=True
)

test_loader = DataLoader(
    TradingDataset(X_test_scaled, y_test),
    batch_size=32
)

## 6. Model Training and Evaluation

In [ ]:
# Training loop with visualization
def train_and_evaluate():
    model = TradingModel(input_size=X_train.shape[1])
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters())
    
    train_losses = []
    val_losses = []
    
    for epoch in range(100):
        # Training code here
        pass
    
    return model, train_losses, val_losses

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.legend()
plt.show()

## 7. Model Analysis and Testing

In [ ]:
# Analyze predictions
def analyze_predictions(model, test_loader):
    predictions = []
    actuals = []
    
    with torch.no_grad():
        for X, y in test_loader:
            pred = model(X)
            predictions.extend(pred.numpy())
            actuals.extend(y.numpy())
    
    return np.array(predictions), np.array(actuals)

# Calculate metrics and create visualizations

## 8. Save Model and Preprocessing Pipeline

In [ ]:
# Save model and scaler
torch.save({
    'model_state_dict': model.state_dict(),
    'scaler': scaler,
    'feature_names': features_df.columns.tolist()
}, '../models/trading_model.pth')